# Problema de Designação de Pessoas para Instrumentos e Vozes em Dias Específicos

## Conjuntos
- **i**: Conjunto de pessoas.
- **j**: Conjunto de instrumentos.
- **k**: Conjunto de dias.
- **l**: Conjunto de vozes.

## Parâmetros e Variáveis
- **P(i, j)**: Matriz binária indicando se a pessoa **i** pode tocar o instrumento **j**.
- **V(i, l)**: Matriz binária indicando se a pessoa **i** pode cantar na posição **l**.
- **L(j)**: Peso de cada instrumento **j** (variando de 0 a 10).
- **LV(l)**: Peso de cada voz **l** (variando de 0 a 10).
- **D['Peso'][k]**: Peso de cada dia **k** (variando de 0 a 10).
- **Disp(i, k)**: Disponibilidade da pessoa **i** no dia **k**.
- **DPM['Quantidade'][i]**: Quantidade de dias que a pessoa **i** se ofereceu para ajudar por mês.

### Variável de Decisão
- **X(i, j, k, l)**: Variável binária indicando se a pessoa **i** toca o instrumento **j** e canta a voz **l** no dia **k**.

## Função Objetivo
A função objetivo maximiza a eficácia da alocação de pessoas para tocar instrumentos e cantar, levando em consideração a disponibilidade, habilidades e os pesos atribuídos aos dias, instrumentos e vozes. A fórmula matemática é:

Max Somatório{i,j,k,l} X(i,j,k,l) * Disp(i,k) * P(i,j) * L(j) * D['Peso'][k] + X(i,j,k,l) * Disp(i,k) * V(i,l) * LV(l) * D['Peso'][k]

## Restrições
1. **Cada pessoa canta somente uma voz por dia**:
   - Uma pessoa não pode ser alocada para cantar mais de uma voz em um dia. 
  
      Somatório{j,l} X(i,j,k,l) <= 1, ∀i, ∀k

2. **Cada voz é cantada por apenas uma pessoa por dia**:
   - Uma voz específica só pode ser cantada por uma única pessoa em um dia.
   
      Somatório{i,j} X(i,j,k,l) * vetor[l] <= 1, ∀l, ∀k

3. **Cada instrumento é tocado por uma pessoa por dia**:
   - Cada instrumento deve ser tocado por uma única pessoa em um dia.
   
      Somatório{i,l} X(i,j,k,l) * vetor\_inst[j] <= 1, ∀j, ∀k

4. **Cada pessoa toca somente um instrumento por dia**:
   - Uma pessoa só pode tocar um instrumento em um dia.
   
      Somatório{j,l} X(i,j,k,l) <= 1, ∀i, ∀k

5. **Cada pessoa toca somente o número de dias que se ofereceu por mês**:
   - A quantidade de dias que uma pessoa se ofereceu para tocar não pode ser excedida.
   
      Somatório{j,l} (X(i,j,k,l) + X(i,j,k+1,l) + X(i,j,k+2,l) + X(i,j,k+3,l)) <= DPM['Quantidade'][i], ∀i, ∀k \in \{0, \dots, |D.index|-4\}

6. **Pessoa não será alocada se não tocar o instrumento específico**:
   - Se uma pessoa não sabe tocar um instrumento específico, ela não será alocada para tocá-lo.
   
      X(i,j,k,l) = 0, se P(i,j) = 0, ∀i, ∀j, ∀k, ∀l

### Descrição do Modelo
   O modelo busca alocar pessoas para tocar instrumentos e cantar em determinadas vozes nos dias disponíveis, maximizando a utilização das suas habilidades e disponibilidades. A função objetivo valoriza as alocações que melhor utilizam a disponibilidade das pessoas e os pesos atribuídos aos dias, instrumentos e vozes. As restrições garantem uma alocação eficiente e justa, respeitando as capacidades e disponibilidades de cada pessoa, e assegurando que cada recurso (instrumento, voz) seja utilizado adequadamente.



In [6]:
import pandas as pd

file_name = 'Dados Louvor Shalom.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Peso dos Dias','Disponibilidade', 'Dias por Mês', 'Vozes', 'Peso das Vozes'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')
V = sheets_dict['Vozes'].set_index('Pessoas')

L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
LV = sheets_dict['Peso das Vozes'].set_index('Voz').squeeze()

D = sheets_dict['Peso dos Dias'].set_index('Dias')

Disp = sheets_dict['Disponibilidade'].set_index('Dias')
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')

# Convertendo os índices para o tipo datetime
D.index = pd.to_datetime(D.index)
Disp.index = pd.to_datetime(Disp.index)

# Formatando os índices para o formato desejado
D.index = D.index.strftime('%d_%m_%Y')
Disp.index = Disp.index.strftime('%d_%m_%Y')

vetor = pd.Series(1, index=V.columns)
vetor['Nada'] = 0

vetor_inst = pd.Series(1, index=P.columns)
vetor_inst['Nenhum'] = 0

In [13]:
import pandas as pd
from pulp import LpVariable, LpProblem, lpSum, LpMaximize, value

# Definindo a variável X como uma variável de 4 dimensões (i, j, k, l)
X = LpVariable.dicts("X", (P.index, P.columns, D.index, V.columns), cat='Binary')

# Criando o problema de otimização
prob = LpProblem("Problem", LpMaximize)

# Função objetivo atualizada
prob += lpSum([
    X[i][j][k][l] * Disp[i][k] * P.loc[i, j] * L[j] * D['Peso'][k] + X[i][j][k][l] * Disp[i][k] * V.loc[i, l] * LV[l]*D['Peso'][k]
    for i in P.index
    for j in P.columns
    for k in D.index
    for l in V.columns
])

# Cada pessoa canta somente uma voz por dia
for i in P.index:  # para cada pessoa
    for k in D.index:  # para cada dia
        prob += lpSum([X[i][j][k][l] for j in P.columns for l in V.columns]) <= 1

# Cada voz é cantada por apenas uma pessoa por dia
for l in V.columns:  # para cada voz
    for k in D.index:  # para cada dia
        prob += lpSum([X[i][j][k][l]*vetor[l] for i in P.index for j in P.columns]) <= 1

# Um instrumento só é tocado por uma pessoa em cada dia K
for j in P.columns:  # para todo instrumento
    for k in D.index:  # para todo dia
        prob += lpSum([X[i][j][k][l]*vetor_inst[j] for i in P.index for l in V.columns]) <= 1

# Cada pessoa toca somente um instrumento cada dia
for i in P.index:  # cada pessoa
    for k in D.index:  # para todo dia
        prob += lpSum([X[i][j][k][l] for j in P.columns for l in V.columns]) <= 1

# Cada pessoa toca somente o número de dias que se ofereceu por mês
for i in P.index: 
    for k in range(len(D.index) - 3):
        prob += lpSum([
            X[i][j][D.index[k]][l] + X[i][j][D.index[k+1]][l] + X[i][j][D.index[k+2]][l] + X[i][j][D.index[k+3]][l]
            for j in P.columns for l in V.columns
        ]) <= DPM['Quantidade'][i]

# A pessoa não será alocada se não tocar o instrumento específico
for i in P.index:  # pessoas
    for j in P.columns:  # instrumentos
        for k in D.index:  # dias
            for l in V.columns:  # vozes
                if P.loc[i, j] == 0:  # tabela de capacidades. Pessoa i toca o instrumento j.
                    prob += X[i][j][k][l] == 0

# Resolver o problema
prob.solve()

# Tabela de vozes por dia
vozes_por_dia = pd.DataFrame(index=D.index, columns=V.columns)

for k in D.index:
    for l in V.columns:
        for i in P.index:
            for j in P.columns:
                if value(X[i][j][k][l]) == 1:
                    vozes_por_dia.at[k, l] = i

# Tabela de instrumentos por dia
instrumentos_por_dia = pd.DataFrame(index=D.index, columns=P.columns)

for k in D.index:
    for j in P.columns:
        for i in P.index:
            for l in V.columns:
                if value(X[i][j][k][l]) == 1:
                    instrumentos_por_dia.at[k, j] = i

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/henrique/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/731e1c2076114229ae49613d58c2e65b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/731e1c2076114229ae49613d58c2e65b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5447 COLUMNS
At line 80014 RHS
At line 85457 BOUNDS
At line 92868 ENDATA
Problem MODEL has 5442 rows, 7410 columns and 54338 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3205 - 0.03 seconds
Cgl0002I 4615 variables fixed
Cgl0004I processed model has 554 rows, 2795 columns (2795 integer (2795 of which binary)) and 15191 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 89 integers unsatisfied sum - 28.3333
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -2947 iterations 495
Cbc0038I Sol

In [10]:
vozes_por_dia.fillna('', inplace=True)
vozes_por_dia.drop(columns=['Nada'])

,Ministração,Voz1,Voz2,Voz3
Dias,,,,
08_06_2024,Henrique,Tassi,Rafael,Dani
15_06_2024,Ricardo,Marcela,Rafael,Ariane
22_06_2024,Henrique,Tassi,Everlyn,Ariane
29_06_2024,Ricardo,Marcela,Dani,Júlia
06_07_2024,Henrique,Rafael,Júlia,Tassi
13_07_2024,Rafael,Ricardo,Ariane,Marcela
20_07_2024,Henrique,Tassi,Ariane,Everlyn
27_07_2024,Ricardo,Dani,Marcela,Júlia
03_08_2024,Rafael,Everlyn,Dani,Henrique


In [12]:
instrumentos_por_dia.fillna('', inplace=True)
instrumentos_por_dia.drop(columns=['Nenhum'])

,Bateria,Baixo,Teclado,Violão,Guitarra
Dias,,,,,
08_06_2024,Israel,Gabriel,Henrique,Rafael,Osvaldo
15_06_2024,Marcela,Ricardo,Vinicius,Rafael,Daniel
22_06_2024,Mikael,Gabriel,Bernardo,Henrique,Enzo
29_06_2024,Israel,Mikael,Mauri,Ricardo,Osvaldo
06_07_2024,Lucas,Gabriel,Henrique,Rafael,Osvaldo
13_07_2024,Lucas,Rafael,Vinicius,Ricardo,Daniel
20_07_2024,Mikael,Gabriel,Bernardo,Henrique,Enzo
27_07_2024,Israel,Mikael,Mauri,Ricardo,Osvaldo
03_08_2024,Lucas,Mikael,Henrique,Rafael,Enzo
